In [39]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## CATS permits

Contains information on current boiler and industrial operation permits in New York City registered with DEP.
* Looking for ways to join boiler info to records from inspections, both have BIN numbers, but many bins have more than one boiler

In [ ]:
df_CATS = pd.read_csv('data/CATS_Permits.csv', 
                      parse_dates=['ExpirationDate', 'IssueDate'])

In [1]:
df_CATS.head(20)

NameError: name 'df_CATS' is not defined

In [22]:
df_CATS.iloc[0]

RequestID                                             256924
ApplicationID                                       CA000898
RequestType                                     REGISTRATION
House                                                    420
Street                                       RIVERSIDE DRIVE
Borough                                            MANHATTAN
Bin                                                    57365
Block                                                   1896
Lot                                                        1
OwnerName         420 RIVERSIDE OWNERS CORP-ATT:HOFFMAN MGMT
ExpirationDate                           2022-01-22 00:00:00
Make                                                   LAARS
Model                                                PW-1200
BurnerMake                                             LAARS
BurnerModel                                          PW-1200
PrimaryFuel                                       NATURALGAS
SecondaryFuel           

In [33]:
df_safety_boiler.groupby(['Bin Number'])['Boiler ID'].nunique().sort_values(ascending=False).head()

Bin Number
3347532    70
3071219    45
4112321    40
4092120    39
2005898    37
Name: Boiler ID, dtype: int64

#### Bin 3347532 is Brooklyn Army Terminal, made up of 16 buildings

In [47]:
df_safety_boiler.iloc[0]

Tracking Number             2017-10000008199Y0001-24915
Boiler ID                              10000008199Y0001
Report Type                                     Initial
ApplicantFirst Name                                DREW
Applicant Last Name                              THOMAS
Applicant License Type                               BB
Applicant License Number                           5468
Owner First Name                                BARBARA
Owner Last Name                                KAMMERER
Boiler Make                               ROCKMILLS STE
Boiler Model                                        NaN
Pressure Type                              Low Pressure
Inspection Type                                External
Inspection Date                     11/17/2017 00:00:00
Defects Exist                                        No
LFF (45 Days)                                         0
LFF (180 Days)                                        0
Filing Fee                                      

## Joining building information to Boiler
First join Bin to pluto through PAD

In [48]:
pad = pd.read_csv('data/pad/bobaadr.txt')
pad.iloc[0]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


boro                                          1
block                                         1
lot                                          10
bin                                     1089249
lhnd                                           
lhns                                           
lcontpar                                       
lsos                                          L
hhnd                                           
hhns                                           
hcontpar                                       
hsos                                          L
scboro                                        1
sc5                                          74
sclgc                                         1
stname         HUGH CAREY TNNL VENTILATOR BLDG 
addrtype                                      N
realb7sc                                       
validlgcs                              01      
dapsflag                                       
naubflag                                

In [49]:
pluto = pd.read_csv('data/pluto/pluto_18v2_1.csv', usecols=usecols)
pluto.iloc[0]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (16,17,18,20,22,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


borough                                   BX
block                                   5641
lot                                      670
cd                                       210
ct2010                                   516
cb2010                                     2
schooldist                               NaN
council                                   13
zipcode                                  NaN
firecomp                                 NaN
policeprct                               NaN
healtharea                               NaN
sanitboro                                NaN
sanitsub                                    
address                   CITY ISLAND AVENUE
zonedist1                                NaN
zonedist2                                NaN
zonedist3                                NaN
zonedist4                                NaN
overlay1                                 NaN
overlay2                                 NaN
spdist1                                  NaN
spdist2   

In [42]:
pluto.columns

Index(['borough', 'block', 'lot', 'cd', 'ct2010', 'cb2010', 'schooldist',
       'council', 'zipcode', 'firecomp', 'policeprct', 'healtharea',
       'sanitboro', 'sanitsub', 'address', 'zonedist1', 'zonedist2',
       'zonedist3', 'zonedist4', 'overlay1', 'overlay2', 'spdist1', 'spdist2',
       'spdist3', 'ltdheight', 'splitzone', 'bldgclass', 'landuse',
       'easements', 'ownertype', 'ownername', 'lotarea', 'bldgarea', 'comarea',
       'resarea', 'officearea', 'retailarea', 'garagearea', 'strgearea',
       'factryarea', 'otherarea', 'areasource', 'numbldgs', 'numfloors',
       'unitsres', 'unitstotal', 'lotfront', 'lotdepth', 'bldgfront',
       'bldgdepth', 'ext', 'proxcode', 'irrlotcode', 'lottype', 'bsmtcode',
       'assessland', 'assesstot', 'exemptland', 'exempttot', 'yearbuilt',
       'yearalter1', 'yearalter2', 'histdist', 'landmark', 'builtfar',
       'residfar', 'commfar', 'facilfar', 'borocode', 'bbl', 'condono',
       'tract2010', 'xcoord', 'ycoord', 'zonemap', '

In [46]:
pad.columns

Index(['boro', 'block', 'lot', 'bin', 'lhnd', 'lhns', 'lcontpar', 'lsos',
       'hhnd', 'hhns', 'hcontpar', 'hsos', 'scboro', 'sc5', 'sclgc', 'stname',
       'addrtype', 'realb7sc', 'validlgcs', 'dapsflag', 'naubflag', 'parity',
       'b10sc', 'segid', 'zipcode', 'physical_id'],
      dtype='object')

In [45]:
[x for x in pluto.columns.values if x in pad.columns.values]

['block', 'lot', 'zipcode']

### need to create a new bbl to join two dataframes

In [55]:
# – 1 (Manhattan), 2 (Bronx),
# 3 (Brooklyn), 4 (Queens), or 5 (Staten Island). The second and third characters are the
# Section Number and the fourth and fifth characters are the Volume Numb
borodict = {'BX': 2, 'QN': 4, 'SI': 5, 'MN': 1, 'BK': 3}

In [75]:
pluto['boro'] = pluto.borough.map(borodict).dropna(axis=0)
# pluto['boro'] = pluto[pluto.boro < 6].reset_index(drop=True)
# pluto['boro'] = pluto.boro.astype(int)
pluto.boro.head()

0    2.0
1    4.0
2    2.0
3    2.0
4    5.0
Name: boro, dtype: float64

In [78]:
pluto = pluto[pluto.boro < 6].reset_index(drop=True)

In [87]:
pluto['boro'] = pluto.boro.astype(int)
pluto['block'] = pluto.block.astype(int)
pluto['lot'] = pluto.lot.astype(int)

In [88]:
pluto['joinbbl'] = pluto.boro.astype(str) + pluto.block.astype(str) + pluto.lot.astype(str)

In [89]:
pluto.joinbbl.head()

0    25641670
1      439353
2     2420541
3     2240214
4       56383
Name: joinbbl, dtype: object

In [ ]:
pad['joinbbl'] = pad.boro.astype(str) + pad.block.astype(str) + pad.lot.astype(str)

In [96]:
usecols = ['borough', 'block', 'lot', 'cd', 'ct2010', 'cb2010', 'zipcode', 'healtharea',
       'address', 'bldgclass', 'landuse', 'ownertype', 'ownername', 'bldgarea', 'comarea',
       'resarea', 'officearea', 'retailarea', 'garagearea', 'strgearea',
       'factryarea', 'otherarea', 'numbldgs', 'numfloors',
       'unitsres', 'unitstotal', 'lottype', 'yearbuilt',
       'yearalter1', 'yearalter2', 'histdist', 'landmark', 'builtfar',
       'residfar', 'commfar', 'facilfar', 'borocode', 'bbl', 'condono',
       'tract2010', 'xcoord', 'ycoord', 'taxmap', 'boro', 'joinbbl']

In [98]:
pluto = pluto[usecols]

In [99]:
pad.iloc[0]

boro                                          1
block                                         1
lot                                          10
bin                                     1089249
lhnd                                           
lhns                                           
lcontpar                                       
lsos                                          L
hhnd                                           
hhns                                           
hcontpar                                       
hsos                                          L
scboro                                        1
sc5                                          74
sclgc                                         1
stname         HUGH CAREY TNNL VENTILATOR BLDG 
addrtype                                      N
realb7sc                                       
validlgcs                              01      
dapsflag                                       
naubflag                                

In [100]:
test = pad.merge(pluto, left_on='joinbbl', right_on='joinbbl')
test.head()

,boro_x,block_x,lot_x,bin,lhnd,lhns,lcontpar,lsos,hhnd,hhns,...,commfar,facilfar,borocode,bbl,condono,tract2010,xcoord,ycoord,taxmap,boro_y
0,1,1,10,1089249,,,,L,,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1
1,1,1,10,1000000,,,,R,,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1
2,1,1,10,1000000,,,,R,,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1
3,1,1,10,1000000,,,,R,,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1
4,1,1,10,1000000,,,,R,,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1


In [101]:
test.to_csv('data/Buildings_w_Pluto.csv')